In [ ]:
%pip install mlxtend --upgrade

In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
import time
import re
import itertools
import copy

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Question 1
#### Test drive the basic version of FP Growth algorithms for Frequent Itemset Mining using the package /
library support in the platform of your choice. Test it with various support and confidence measures and
generate a time comparison for varied data set sizes. To do the performance comparison you may use
benchmark datasets provided for FIM such as the FIMI workshop or other sources.

In [3]:
data=pd.read_csv("store_data.csv", header=None)
df=pd.DataFrame(data)


#creating list
records = [[y for y in x if pd.notna(y)] for x in df.values.tolist()] 

# Convert the data to a transaction database
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Perform FP-Growth algorithm
start = time.process_time()
frequent_itemsets = fpgrowth(df, min_support=0.005, use_colnames=True)
end = time.process_time()

# Print the results
print("Frequent itemsets:")
print(frequent_itemsets)

print("\n Time Taken:",end-start)

Frequent itemsets:
      support                          itemsets
0    0.238368                   (mineral water)
1    0.132116                       (green tea)
2    0.076523                  (low fat yogurt)
3    0.071457                          (shrimp)
4    0.065858                       (olive oil)
..        ...                               ...
720  0.005999               (eggs, french wine)
721  0.005199          (chocolate, french wine)
722  0.005333          (mineral water, carrots)
723  0.005733  (escalope, mushroom cream sauce)
724  0.005066      (mineral water, nonfat milk)

[725 rows x 2 columns]

 Time Taken: 0.26431953500000027


## Question 2
#### Extend the Apriori Algorithm discussed in the class supporting Transaction Reduction approach to improve the time complexity issue as a result of the repeated scans limitation of Apriori. You may compare this extended version with the earlier implementations in (1) over the same benchmark dataset.

In [4]:
data=pd.read_csv("store_data.csv", header=None)
df=pd.DataFrame(data)


#creating list
records = [[y for y in x if pd.notna(y)] for x in df.values.tolist()]   
print(records[10])

['eggs', 'pet food']


In [5]:
# creating the database with tranjaction number
Database={}
for i in range(len(records)):
    Database["T"+str(i+1)]=records[i]


# creating C1(itemset with one element) with the count
Itemset={}
for i in range(len(records)):
    for j in range(len(records[i])):
        if(frozenset([records[i][j]]) not in Itemset):            # Frozen set is just an immutable version of a Python set object. While elements of a set can be modified at 
            Itemset[frozenset([records[i][j]])]=1                 # any time, elements of the frozen set remain the same after creation.
        else:
            Itemset[frozenset([records[i][j]])]+=1
print(Itemset)

{frozenset({'shrimp'}): 536, frozenset({'almonds'}): 153, frozenset({'avocado'}): 250, frozenset({'vegetables mix'}): 193, frozenset({'green grapes'}): 68, frozenset({'whole weat flour'}): 70, frozenset({'yams'}): 86, frozenset({'cottage cheese'}): 239, frozenset({'energy drink'}): 200, frozenset({'tomato juice'}): 228, frozenset({'low fat yogurt'}): 574, frozenset({'green tea'}): 991, frozenset({'honey'}): 356, frozenset({'salad'}): 37, frozenset({'mineral water'}): 1788, frozenset({'salmon'}): 319, frozenset({'antioxydant juice'}): 67, frozenset({'frozen smoothie'}): 475, frozenset({'spinach'}): 53, frozenset({'olive oil'}): 494, frozenset({'burgers'}): 654, frozenset({'meatballs'}): 157, frozenset({'eggs'}): 1348, frozenset({'chutney'}): 31, frozenset({'turkey'}): 469, frozenset({'milk'}): 972, frozenset({'energy bar'}): 203, frozenset({'whole wheat rice'}): 439, frozenset({'whole wheat pasta'}): 221, frozenset({'french fries'}): 1282, frozenset({'soup'}): 379, frozenset({'light cre

In [6]:
Min_Sup_Count=0.005*len(records)
Li={}
# filtering Itemset whose support is greater than min support
for key,val in Itemset.items():
    if(val>=Min_Sup_Count):
        Li[key]=val
Itemset=Li


# get the support count(occuring) of the set of items in the Database
def check(miniset,Database):
    count=0
    for key,val in Database.items():
        if(frozenset(val).intersection(miniset)==miniset):
            count+=1
    return count


# to get C_(i+1) from L_i
def get_c(Li,Database,cur_sot):
    c={}
    for key,vals in Li.items():
        for key1,vals1 in Li.items():
            if (key1!=key):
                miniset=key1.union(key)
                if(len(miniset)>cur_sot):
                    continue
                count=check(miniset,Database)
                c[miniset]=count
    return c


# to get L_i from C_i
def get_l(c,Min_Sup_Count):
    rem_keys=[]
    for key,val in c.items():
        if(val<Min_Sup_Count):
            rem_keys.append(key)
    for key in rem_keys:
        c.pop(key)
    return c


# this is where we reduce the database. We remove those tranjaction whose size is less
def remove_transaction(Database,sot):
    rem_keys=[]
    for key,val in Database.items():
        if(len(val)<sot):
            rem_keys.append(key)
    for key in rem_keys:
        Database.pop(key)
    return Database


# Item which could not come into current L_i (because their support is less than min support) remove those items from database
def remove_item(Li,Database):
    miniset=set()
    for key,val in Li.items():
        miniset=miniset.union(key)
    for key,val in Database.items():
        Database[key]=list(set(val) & miniset)
        
    return Database

In [7]:
start = time.process_time()

Final_List=[]
sot=1
final_c={}
while(1):
    Database=remove_transaction(Database,sot)
    c=get_c(Li,Database,sot+1)
    sot+=1
    
    Li=get_l(c,Min_Sup_Count)
    Database=remove_item(Li,Database)
    if(len(Li)==0):
        break
    else:
        final_c=Li

end = time.process_time()

print("Total iteration: " + str(sot))
print("min_support: " + str(Min_Sup_Count))
print("\n Time Taken with transjaction reduction  "+str(end-start)+" seconds")

Total iteration: 4
min_support: 37.505

 Time Taken with transjaction reduction  122.93064487400001 seconds


## Question 3
#### Test drive any one implementation in (1) or (2) adopting a Vertical Transaction Database format.

In [9]:
records=[[100,400,500,700,800,900],[100,200,300,400,600,800,900],[300,500,600,700,800,900],[200,400],[100,800]]


Database={}
for i in range(len(records)):
    Database["T"+str(i+1)]=records[i]

Itemset={}
for i in range(len(records)):
    for j in range(len(records[i])):
        if(frozenset([records[i][j]]) not in Itemset):
            Itemset[frozenset([records[i][j]])]=1
        else:
            Itemset[frozenset([records[i][j]])]+=1



# creating  virtical format where each row will contain those tranjaction numbers where that item set is present
Database_vdf={}
for key,val in Database.items():
    for x in val:
        if(frozenset([x]) not in Database_vdf):
            Database_vdf[frozenset([x])]=frozenset([key])
        else:
            Database_vdf[frozenset([x])]=frozenset([key]).union(Database_vdf[frozenset([x])])


records_vdf=[]
for key,val in Database_vdf.items():
    records_vdf.append(val)


te = TransactionEncoder()
te_ary = te.fit(records_vdf).transform(records_vdf)
df_vdf = pd.DataFrame(te_ary, columns=te.columns_)


start = time.process_time()

print(apriori(df_vdf, min_support=0.003,use_colnames=True))

time_taken=time.process_time() - start
print("\n Time Taken for Mining using Apriori =",time_taken)

     support          itemsets
0   0.666667              (T1)
1   0.777778              (T2)
2   0.666667              (T3)
3   0.222222              (T4)
4   0.222222              (T5)
5   0.444444          (T1, T2)
6   0.444444          (T1, T3)
7   0.111111          (T4, T1)
8   0.222222          (T5, T1)
9   0.444444          (T3, T2)
10  0.222222          (T4, T2)
11  0.222222          (T5, T2)
12  0.111111          (T5, T3)
13  0.222222      (T3, T1, T2)
14  0.111111      (T4, T1, T2)
15  0.222222      (T5, T1, T2)
16  0.111111      (T5, T1, T3)
17  0.111111      (T5, T3, T2)
18  0.111111  (T5, T3, T1, T2)

 Time Taken for Mining using Apriori = 0.020376107999993565


## Question 4
#### Using a vertical transaction database notation, generate the FI’s following the intersection approach (basic ECLAT) discussed in the class. Use earlier benchmark datasets in (1).

In [10]:
data=pd.read_csv("store_data.csv", header=None)
df=pd.DataFrame(data)

records = [[y for y in x if pd.notna(y)] for x in df.values.tolist()]

Database={}
for i in range(len(records)):
    Database["T"+str(i+1)]=records[i]


# Scan the Dataset to convert to vertical form
Database_vdf={}
for key,val in Database.items():
    for x in val:
        if(frozenset([x]) not in Database_vdf):
            Database_vdf[frozenset([x])]=frozenset([key])
        else:
            Database_vdf[frozenset([x])]=frozenset([key]).union(Database_vdf[frozenset([x])])


# to remove items less than min support
def remove_items_vdf(Database_vdf,Min_Sup):
    rem_keys=[]
    for key,val in Database_vdf.items():
        if(len(val)<Min_Sup):
            rem_keys.append(key)
    for key in rem_keys:
        Database_vdf.pop(key)
    return Database_vdf


def get_C(Ci,iteration):
    New_Ci={}
    for key1,val1 in Ci.items():
        for key2,val2 in Ci.items():
            if(key1!=key2):
                new_key=key1.union(key2)
                if(len(new_key)>iteration):
                    continue
                else:
                    New_Ci[new_key]=val1.intersection(val2)
    return New_Ci





start = time.process_time()
Min_Sup_Count_vdf=0.005*len(records)

Li=remove_items_vdf(Database_vdf,Min_Sup_Count_vdf)
count=1
while(1):
    count+=1
    c=get_C(Li,count)
    Li=remove_items_vdf(c,Min_Sup_Count_vdf)
    if(len(Li)==0):
        break
    else:
        final_vdf=Li

end = time.process_time()
print("Total iteration: " + str(count-1))
print("min_support: " + str(Min_Sup_Count_vdf))
print("Time Taken with transjaction reduction  "+str(end-start)+" seconds")

Total iteration: 3
min_support: 37.505
Time Taken with transjaction reduction  0.29762266400001636 seconds


## Question 5
#### Extend the basic Apriori algorithm to generate Frequent Patterns which differentiate ab from ba (ordered patterns generation).

In [11]:
# to get C_(i+1) form L_i
def get_c(l0,sequences,cur_sot):
    c1={}
    for key1 in l0.keys():
        for key2 in l0.keys():
            if(key1!=key2):
                new_key=key1+key2
                if(len(new_key)==cur_sot):
                    # below part will get the support count of the new_key
                    for key in sequences.keys():
                        if(new_key not in c1):
                            c1[new_key]=len(re.findall(new_key,sequences[key]))
                        else:
                            c1[new_key]+=len(re.findall(new_key,sequences[key]))
    return c1
                    

# To get L_i from C_i                   
def get_l(c,Min_Sup_Count):
    Li={}
    for key in c.keys():
        if(c[key]>=Min_Sup_Count):
            Li[key]=c[key]
    return Li

In [12]:
Transaction={1: 'aabcbacdcf', 2: 'adcabacae', 3: 'efabadfcb', 4: 'egbafcbc'}


c0={}
for key in Transaction.keys():
    for x in Transaction[key]:
            if(x not in c0):
                c0[x]=1
            else:
                c0[x]+=1

l0={}
Min_Sup_Count=1
for key in c0.keys():
    if(c0[key]>=Min_Sup_Count):
        l0[key]=c0[key]


sot=1
final_sequence={}
Min_Sup_Count=3
while(1):
    c=get_c(l0,Transaction,sot+1)
    sot+=1
    Li=get_l(c,Min_Sup_Count)
    if(len(Li)==0):
        break
    else:
        final_sequence=Li


print("Total iteration: " + str(sot-1))
print("Frequent Patterns are:",[x for x in final_sequence.keys()] )
# int the output we can see both ab and ba

Total iteration: 2
Frequent Patterns are: ['ab', 'ba', 'cb']


## Question 6
#### Implement following extensions to Apriori Algorithm (discussed / to be discussed in the class): Hash based strategy, Partitioning Approach. You may refer to online tutorials for a formal pseudocode description.

#### -> Hash based

In [13]:
transactions = [{'A','C','D'},{'B','C','E'},{'A','B','C','E'},{'B','E'}]

database_hash={}
count=0
for i in transactions:
    count+=1
    database_hash["T"+str(count)]=i

c0={}
for i in transactions:
    for j in i:
        if(j in c0):
            c0[j]+=1
        else:
            c0[j]=1
      
c0

{'A': 2, 'C': 3, 'D': 1, 'B': 3, 'E': 3}

In [14]:
Min_Sup_Count=2
rem_keys=[]
Li={}
for key,val in c0.items():
    if(val>=Min_Sup_Count):
        Li[key]=val
        rem_keys.append(key)

Li

{'A': 2, 'C': 3, 'B': 3, 'E': 3}

In [15]:
# creating combinations of two elements for each transaction
for key,val in database_hash.items():
    val=[set(i) for i in itertools.combinations(val, 2)]
    sets=[]
    for i in range(len(val)):
        sets.append(sorted(val[i]))

    database_hash[key]=sets
database_hash

{'T1': [['A', 'C'], ['A', 'D'], ['C', 'D']],
 'T2': [['B', 'C'], ['B', 'E'], ['C', 'E']],
 'T3': [['A', 'B'],
  ['A', 'C'],
  ['A', 'E'],
  ['B', 'C'],
  ['B', 'E'],
  ['C', 'E']],
 'T4': [['B', 'E']]}

In [16]:
# order will be used to generate hash table
order={}
count=0
for key in sorted(c0.keys()):
    count+=1
    order[key]=count
order

{'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}

In [17]:
#Generate hash table
Hash_Table={}
for key,items in database_hash.items():
    for x in items:
        val=(order[x[0]]*10+order[x[1]])%7
        if(val in Hash_Table):
            Hash_Table[val].append(x)
        else:
            Hash_Table[val]=[x]
Hash_Table

{6: [['A', 'C'], ['C', 'D'], ['A', 'C']],
 0: [['A', 'D'], ['C', 'E'], ['C', 'E']],
 2: [['B', 'C'], ['B', 'C']],
 4: [['B', 'E'], ['B', 'E'], ['B', 'E']],
 5: [['A', 'B']],
 1: [['A', 'E']]}

In [18]:
#Generate C2 using hash table
C2={}
keys=sorted(Li.keys())
for i in range(len(keys)):
    for j in range(i+1,len(keys)):
        New_key=[keys[i], keys[j]]
        New_val=(order[keys[i]]*10+order[keys[j]])%7
        if(len(Hash_Table[New_val]) >= Min_Sup_Count):                          # if total item count in the hash table in that index is less than min support count 
            C2[frozenset(New_key)]=Hash_Table[New_val].count(New_key)           # then it is confirmed that support count of that item set must be less than min support count
print(C2)

{frozenset({'A', 'C'}): 2, frozenset({'B', 'C'}): 2, frozenset({'B', 'E'}): 3, frozenset({'C', 'E'}): 2}


In [19]:
#Generate L2
L2={}
for key,val in C2.items():
    if(val>=Min_Sup_Count):
        L2[key]=val

print(L2)

{frozenset({'A', 'C'}): 2, frozenset({'B', 'C'}): 2, frozenset({'B', 'E'}): 3, frozenset({'C', 'E'}): 2}


#### -> Partitioning Approach

In [20]:
data=pd.read_csv("store_data.csv", header=None)
df=pd.DataFrame(data)

records = [[y for y in x if pd.notna(y)] for x in df.values.tolist()]

Database={}
for i in range(len(records)):
    Database["T"+str(i+1)]=records[i]

te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df = pd.DataFrame(te_ary, columns=te.columns_)


# devide the dataset into 13 partitions
dfs=[]
for i in range(0,df.shape[0],int(df.shape[0]/13)):
    dfs.append(df.iloc[i:i+int(df.shape[0]/13)])


# calculating apriori of each partition
results=[]
for i in dfs:
    results.append(apriori(i, min_support=0.01,use_colnames=True))
  
results[0]

,support,itemsets
0,0.025997,(almonds)
1,0.010399,(antioxydant juice)
2,0.029463,(avocado)
3,0.012132,(body spray)
4,0.017331,(brownies)
...,...,...
316,0.012132,"(pasta, mineral water, shrimp)"
317,0.012132,"(soup, mineral water, spaghetti)"
318,0.010399,"(soup, mineral water, tomatoes)"
319,0.010399,"(pasta, shrimp, spaghetti)"


In [21]:
# this is the part where we combine all parts
final_candidate_set={}
for i in results:
    for j in range(i.shape[0]):
        item=i.iloc[j][1]
        if(item in final_candidate_set):
            final_candidate_set[item]+=(i.iloc[j][0]*int(df.shape[0]/13))    # support count = support * number of rows in i
        else:
            final_candidate_set[item]=(i.iloc[j][0]*int(df.shape[0]/13))



final_results={}
Min_Sup_Count=int(df.shape[0]*(0.1))
for key,val in final_candidate_set.items():
    if(val>=Min_Sup_Count):
        final_results[key]=val
final_results

{frozenset({'chocolate'}): 1229.0,
 frozenset({'eggs'}): 1348.0,
 frozenset({'french fries'}): 1282.0,
 frozenset({'green tea'}): 991.0,
 frozenset({'milk'}): 972.0,
 frozenset({'mineral water'}): 1788.0,
 frozenset({'spaghetti'}): 1306.0}

## Question 7
#### Implement the Dynamic Itemset Counting Algorithm for Frequent Itemset Generation.

In [22]:
# to get all subset of length n
def get_subset(S,n):
    a = itertools.combinations(S,n)
    results = []
    for i in a:
        results.append(set(i))
    return(results)


# try to append items in unique_itemset with set S
def get_superset(S,unique_itemset):
    result = []
    a = set()
    for i in unique_itemset:
        # print("get_superset1 ", S, i, i.intersection(S), i.union(S))
        if i.intersection(S)==set():              # check wheather the intersection is null
            a = i.union(S)
            result.append(a)
            a = set()

    return(result)


# check if every possible subset(length less than 1) of Set is subset of frequent_set
def check_subset(Set,frequent_set):
    subset = get_subset(Set,len(Set)-1)
    flag = 1
    temp = []

    for i in frequent_set:
        temp.append(i[0])

    frequent_set = temp
    for i in subset:
        if i not in frequent_set:
            flag=0
            break

    if flag:
        return(True)
    else:
        return(False)


# return itemsets which is set to 1 in the transaction
def get_itemset(T):
    result = set()
    for i in range(len(T)):
        if T[i]!=0:
            result.add(i+1)

    return(result)

In [23]:
database = [[1,1,0],[1,0,0],[0,1,1],[0,0,0]]
unique_itemset =[{1},{2},{3}]
min_supp = 1
M = 2
size = len(database)


DC = []          # Dashed circle: suspected infrequent itemset
DS = []          # Dashed square: suspected frequent itemset
SC = []          # Solid circle: confirmed infrequent itemset
SS = []          # Solid square: confirmed frequent itemset

for i in unique_itemset:
    DC.append([i,0,0])        # [item_ser, support_count, count_to_trace_full_scan]

print("Initial DC:",DC,"\n")

counter = 0
T = []
while len(DC)!=0 or len(DS)!=0:

    for i in range(counter,counter+M):
        index = i%size
        T = get_itemset(database[index])
        print("Transaction :",T)

        for item in DC:
            item[2]+=1
            if item[0].issubset(T):
                item[1]+=1                # increase support count
        for item in DS:
            item[2]+=1
            if item[0].issubset(T):
                item[1]+=1

    for item in copy.copy(DC):
        if(item[1]>=min_supp):        
            DS.append(item)             # suspected infrequent to suspected frequent
            DC.remove(item)

    for item in copy.copy(DS):
        if(item[2]==size):              # if we scaned the full dataset for this item
            SS.append(item)             # suspected frequent to confirmed frequent
            DS.remove(item)

    for item in copy.copy(DC): 
        if(item[2]==size):              # done full scan
            SC.append(item)             # suspected infrequent to confirmed infrequent
            DC.remove(item)

    frequent_set = copy.copy(DS)
    frequent_set.extend(SS)            # frequent_set will contain both suspected and confirmed frequent
    for item in frequent_set:
        S = get_superset(item[0],unique_itemset)      # try to append A/B/C with the current item set
        for i in S:
            if (check_subset(i,frequent_set)):     # check if every possible subset(length less than 1) of i is subset of frequent_set (antimonotonicity property)
                flag=1
                for x in DC:
                    if x[0]==i:
                        flag=0
                for x in DS:
                    if x[0]==i:
                        flag=0
                for x in SC:
                    if x[0]==i:
                        flag=0
                for x in SS:
                    if x[0]==i:
                        flag=0
                if flag:                     # if we don't find the i
                    DC.append([i,0,0])


    counter+=M
    print("DS: ",DS)
    print("DC: ",DC)
    print("SS: ",SS)
    print("SC: ",SC,"\n")

Initial DC: [[{1}, 0, 0], [{2}, 0, 0], [{3}, 0, 0]] 

Transaction : {1, 2}
Transaction : {1}
DS:  [[{1}, 2, 2], [{2}, 1, 2]]
DC:  [[{3}, 0, 2], [{1, 2}, 0, 0]]
SS:  []
SC:  [] 

Transaction : {2, 3}
Transaction : set()
DS:  []
DC:  [[{1, 2}, 0, 2], [{1, 3}, 0, 0], [{2, 3}, 0, 0]]
SS:  [[{1}, 2, 4], [{2}, 2, 4], [{3}, 1, 4]]
SC:  [] 

Transaction : {1, 2}
Transaction : {1}
DS:  []
DC:  [[{1, 3}, 0, 2], [{2, 3}, 0, 2]]
SS:  [[{1}, 2, 4], [{2}, 2, 4], [{3}, 1, 4], [{1, 2}, 1, 4]]
SC:  [] 

Transaction : {2, 3}
Transaction : set()
DS:  []
DC:  []
SS:  [[{1}, 2, 4], [{2}, 2, 4], [{3}, 1, 4], [{1, 2}, 1, 4], [{2, 3}, 1, 4]]
SC:  [[{1, 3}, 0, 4]] 

